In [1]:
import os
import sys
import time
import json

import urllib
import urllib.request
import requests

import pandas as pd
from bs4 import BeautifulSoup

import re

from tqdm import tqdm

In [5]:
data_path = r"C:\Users\Jwp\Desktop\workspace\Final_project\DataScience\Final_Data\movie\moviePoster"
save_path = r"C:\Users\Jwp\Desktop\workspace\Final_project\DataScience\Final_Data\reply_crwaling\naver_reply"
# data_path = r"C:\Users\chongs\Desktop\git\final_project\DataScience\Final_Data\movie\movieInfo\all"
# save_path = r"C:\Users\chongs\Desktop\git\final_project\DataScience\Final_Data\movie\movieReview"
# data_path = r"D:\git_ec\final_project\DataScience\Final_Data\movie\movieInfo\all"
# save_path = r"D:\git_ec\final_project\DataScience\Final_Data\movie\movieReview"

In [8]:
movies = pd.read_csv(os.path.join(data_path, "movie_info_all2.csv"), encoding="utf-8")

In [11]:
dd = movies[movies.movieNm == "컨저링 2"]

In [22]:
dd

,movieCd,movieNm,movieNmEn,prdtYear,openDt,prdtStatNm,repGenre,genre,repNation,nations,...,actors_en,cast,cast_en,companyCd,companyNm,companyNmEn,companyPartNm,watchGradeNm,poster,userRating
4818,20162941,컨저링 2,The Conjuring 2,2016,20160609,개봉,공포(호러),공포(호러),미국,미국,...,"['Vera Farmiga', 'Patrick Wilson', 'Franka Pot...","['', '', '', '', '']","['', '', '', '', '']","['20100043', '20100043']","['워너브러더스 코리아(주)', '워너브러더스 코리아(주)']","['Warner Bros. Korea', 'Warner Bros. Korea']","['배급사', '수입사']",15세이상관람가,https://ssl.pstatic.net/imgmovie/mdi/mit110/11...,8.23


In [13]:
def progress_bar(i, total):
    k = i/total*100
    star = "*"*int(k/2.5)
    print("Processing: {}/{} [{:.1f}%] {}".format(i, total, k, star), end="\r")

In [14]:
def get_naverApi(movieNm, openDt, director):
    
    client_id = "JjcGyzzf5rjyFKNPCTxG"
    client_secret = "MDi_1o76AT"
    encText = urllib.parse.quote(f"{movieNm}")
    url = "https://openapi.naver.com/v1/search/movie?query=" + encText + f"&yearfrom={openDt-1}&yearto={openDt}"
    request = urllib.request.Request(url)

    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)

    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if(rescode == 200):
        html = response.read().decode("utf-8")
        parse = json.loads(html)
                
        if parse["items"] == []:
            link = ""
            
            
        else:
            if len(parse["items"]) > 1:
                for i, item in enumerate(parse["items"]):
                    if director in item["director"]:
                        idx = i
                        break
                    elif item["pubDate"] == str(openDt):
                        idx = i
                        break
                    else:
                        idx = 0
            else:
                 idx = 0
            
            info = parse["items"][idx]
            link = info["link"]
            
    else:
        link = ""
    
    return movieNm, link

In [15]:
def load_naverMovie(movie_list):
    movie_list.openDt = movie_list.openDt.astype(str)
    ls = movie_list[["movieCd","movieNm", "openDt", "directors"]].values
    
    i = 1
    total = len(ls)
    links = []
    movieNms = []
    movieCds = []
    for v in ls:
#         print(v[1], int(v[2][:4]), v[3].split(" ")[0].split(",")[0][:2])
        movieNm, link = get_naverApi(v[1], int(v[2][:4]), v[3].split(" ")[0].split(",")[0][:2])
        code = v[0]
        links.append(link)
        movieNms.append(movieNm)
        movieCds.append(code)
        progress_bar(i, total)
        i += 1
        time.sleep(0.1)

    print()
    print("complete!")
    
    return movieCds, movieNms, links

In [16]:
def get_reviews(naver_movie_code):
    target_url = f'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code={naver_movie_code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false'
    soup = BeautifulSoup(urllib.request.urlopen(target_url).read(), "html.parser")
    review_total = soup.find("div",class_="score_total")
    
#     review가 없는 경우는 아무것도 리턴하지 않는다
    if review_total is not None:
        number = review_total.find('em').text
        number = int(re.sub(',', '', number))
        page_num = number //10

        user_id = []
        review = []
        score = []
        
        for page in range(1, page_num):
            target_url_page = target_url + '&page={}'.format(page)
            soup = BeautifulSoup(urllib.request.urlopen(target_url_page).read(), "html.parser")
            review_divs = soup.find_all('div',class_='score_reple')
            score_divs = soup.find_all('div', class_="star_score")
            for spans in review_divs:
                txt = spans.find('p').text.strip()
                if txt is not None:
                    review.append(txt)
                else:
                    review.append('')
            for spans in review_divs:
                dl = spans.find('dl')
                if dl is not None:
                    user = dl.find('span').text
                    user_id.append(user)
                else:
                    user_id.append('')
            for span in score_divs:
                if span is not None:
                    s = span.find('em').text.strip()
                    score.append(s)
                else:
                    score.append('')
            
        return user_id, review, score
    else:
        return ["", "", ""]

In [17]:
def load_movieReview(movie_list, save_path):
    movieCds, movieNms, links = load_naverMovie(movie_list)
    
    user_id = []
    review = []
    score = []
    for link in tqdm(links):
        code = link[51:]
        ids, reply, star_score = get_reviews(code)
        user_id.append(ids)
        review.append(reply)
        score.append(star_score)
    
    for i in range(len(movie_list)):
        df = pd.DataFrame([user_id[i], review[i], score[i]], index=["user_id", "review", "score"]).T
        file_name = re.sub('[-=+,#/\?:^$@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '_', f"{movieCds[i]}_{movieNms[i]}.csv".replace(" ", ""))
        df.to_csv(os.path.join(save_path, file_name), index=None, encoding="utf-8")

In [18]:
load_movieReview(dd, save_path)

C:\Users\Jwp\Anaconda3\envs\movie-inside\lib\site-packages\pandas\core\generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


Processing: 1/1 [100.0%] ****************************************
complete!


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [01:47<00:00, 107.25s/it]


In [19]:
mm = pd.read_csv(os.path.join(save_path, "20162941_컨저링2.csv"), encoding="utf-8")

In [21]:
mm

,user_id,review,score
0,월령공주(glor****),"평식이, 시답잖게 쎈척은 ...ㅉㅉ",10
1,구일중(slrh****),나 평식이 옆에 앉아서 영화봣는데 평식아 소리좀 그만질러라...남자색히가 ㅉㅉ,10
2,이보람(dlqh****),평식이가누구에요...,10
3,짱구방구뀐다(kgh1****),'시답잖게 겁주기는' 이런게 영호평론가의 멘트면 아무나 평론가 할 수 있는거 아님??,8
4,쿠앤크(zhfl****),평식이 오줌 찌린 게 여기까지 흘렀다....당장 보러간다!!!,9
...,...,...,...
7875,잉잉(dltm****),관람객\n\n\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\...,1
7876,응아니야(dntw****),이게잼?니미ㅋㅋㅋ1편부터 존1ㄴ나 개노잼,1
7877,ses0****,저는 두 번 놀랬는데 남자친구는 세 번 놀랬어요.,1
7878,Guru(jaew****),평점계의 소금쟁이 박평식,1


### 코드 확인

In [82]:
user_id = []
review = []
score = []
for link in tqdm(links):
    code = link[51:]
    ids, reply, star_score = get_reviews(code)
    user_id.append(ids)
    review.append(reply)
    score.append(star_score)

100%|██████████████████████████████████████████████████████████████████████████████████| 1/1 [28:20<00:00, 1700.45s/it]


In [85]:
len(user_id[0]), len(review[0]), len(score[0])

(68580, 68580, 68580)

In [96]:
for i in range(len(movieNms)):
    df = pd.DataFrame([user_id[i], review[i], score[i]], index=["user_id", "review", "score"]).T
    file_name = re.sub('[-=+,#/\?:^$@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '_', f"{movieCds[i]}_{movieNms[i]}.csv".replace(" ", ""))
    df.to_csv(os.path.join(save_path, file_name), index=None, encoding="utf-8")

In [64]:
reviews = []
for i in range(len(user_id)):
    reviews.append(pd.DataFrame([user_id[i], review[i], score[i]], index=["user_id", "review", "score"]).T)

In [72]:
for i in range(reviews):
    reviews[i].to_csv(os.path.join(save_path, f"{movies.iloc[i].movieCd} {movies.iloc[i].movieNm}.csv"), encoding="utf-8")